Importing required Libraries

In [1]:
import numpy as np
import pandas as pd
import re

######### Raw Data Dump

In [2]:
twitter_data = pd.read_excel("Compiled Data (Covid Apr).xlsx")

In [3]:
twitter_data.head(2)

,Id,Timestamp,Source,Retwwet_Count,User_Name,Location,Tweets
0,1244239930341100032,2020-03-29 12:28:10,Twitter Web App,0,Mamaa,NaN,@iam_davidwilson Please how are you guys adjus...
1,1244239930123049984,2020-03-29 12:28:10,Twitter for Android,0,Opemi Dmh,NaN,@LisaMelz First Name: CORONA\nSurname: VIRUS \...


In [4]:
twitter_data = twitter_data.rename(columns = {'Tweets':'tweets'})

In [5]:
twitter_data=pd.DataFrame(twitter_data)

In [6]:
twitter_data.shape

(126043, 7)

In [7]:
twitter_data=twitter_data.drop(columns = 'Location', axis = 1)

In [8]:
twitter_data= twitter_data.drop_duplicates()

In [9]:
twitter_data.shape

(126031, 6)

In [10]:
twitter_data.iloc[:,5] = twitter_data.iloc[:,5].apply(str)

Pre Processing - Removing unwanted Characters

In [11]:
twitter_data.head(2)

,Id,Timestamp,Source,Retwwet_Count,User_Name,tweets
0,1244239930341100032,2020-03-29 12:28:10,Twitter Web App,0,Mamaa,@iam_davidwilson Please how are you guys adjus...
1,1244239930123049984,2020-03-29 12:28:10,Twitter for Android,0,Opemi Dmh,@LisaMelz First Name: CORONA\nSurname: VIRUS \...


Pre Processing
Remove Punctuations

In [12]:
twitter_data.iloc[:,5] = twitter_data.iloc[:,5].str.replace(r'https(\S)+', r' ')
twitter_data.iloc[:,5] = twitter_data.iloc[:,5].str.replace(r'http ...', r' ')

twitter_data.iloc[:,5] = twitter_data.iloc[:,5].str.replace(r'(RT|rt)[ ]*@[ ]*[\S]+',r' ')
twitter_data.iloc[:,5] = twitter_data.iloc[:,5].str.replace(r'@[\S]+',r' ')

twitter_data.iloc[:,5] = twitter_data.iloc[:,5].str.replace('http', '')

In [13]:
twitter_data.head(5)

,Id,Timestamp,Source,Retwwet_Count,User_Name,tweets
0,1244239930341100032,2020-03-29 12:28:10,Twitter Web App,0,Mamaa,Please how are you guys adjusting to realiti...
1,1244239930123049984,2020-03-29 12:28:10,Twitter for Android,0,Opemi Dmh,First Name: CORONA\nSurname: VIRUS \nNickNam...
2,1244239929619680000,2020-03-29 12:28:10,IFTTT,0,Muhammad Iqbal Dar,Liked on YouTube: Corona Conversation with Dad...
3,1244239928919119872,2020-03-29 12:28:10,Twitter for iPhone,0,â™¥Dream 2021 ðŸ’™ Rache â™¥,Letâ€™s fight Corona Virus. Get well soon.
4,1244239928608740096,2020-03-29 12:28:10,Twitter for Android,0,Alaxender0687,Aaj desh mein Corona ko Phelne se Rokna bhut h...


In [14]:
PUNCT_TO_REMOVE = '!"#$%&\'()*+,-./:;<=>?@[\\]^_{|}~`'

In [15]:
def remove_punctuation(text):
    return text.translate(str.maketrans(' ', ' ', PUNCT_TO_REMOVE))

In [16]:
twitter_data.iloc[:,5] = twitter_data.iloc[:,5].apply(lambda text: remove_punctuation(text))

In [17]:
all_words = list(twitter_data.iloc[:,5].str.lower().str.split(' ', expand=True).stack().unique())

In [18]:
twitter_data['tweets'].head(10)

0      Please how are you guys adjusting to realiti...
1      First Name CORONA\nSurname VIRUS \nNickName ...
2    Liked on YouTube Corona Conversation with Dad ...
3             Letâ€™s fight Corona Virus Get well soon
4    Aaj desh mein Corona ko Phelne se Rokna bhut h...
5        I bought one due to Corona FIFA is shit St...
6    First death due to corona Buldana Maharashtra ...
7    How we handle Corona crisis will determine our...
8        This corona virus shit fuckin up my sleep smh
9    Corona doesnt weather you are rich or poverty ...
Name: tweets, dtype: object

Remove most occuring words

In [19]:
from collections import Counter
cnt = Counter()
for text in twitter_data['tweets'].values:
    for word in text.split():
        cnt[word] += 1

In [20]:
cnt.most_common(70)

[('the', 83647),
 ('to', 70909),
 ('of', 47685),
 ('and', 45069),
 ('corona', 39462),
 ('a', 38293),
 ('is', 37105),
 ('in', 36568),
 ('Corona', 31573),
 ('for', 28171),
 ('I', 27100),
 ('this', 20684),
 ('COVID19', 20225),
 ('you', 20019),
 ('on', 18403),
 ('are', 18285),
 ('virus', 17847),
 ('that', 17667),
 ('it', 16984),
 ('with', 15400),
 ('be', 15050),
 ('have', 14399),
 ('my', 12766),
 ('from', 12751),
 ('not', 12602),
 ('we', 11077),
 ('will', 10559),
 ('all', 10279),
 ('at', 9950),
 ('Covid19', 9632),
 ('people', 9526),
 ('The', 9329),
 ('has', 9034),
 ('as', 8797),
 ('but', 8778),
 ('was', 8418),
 ('they', 8103),
 ('by', 7993),
 ('your', 7871),
 ('about', 7854),
 ('so', 7794),
 ('me', 7636),
 ('can', 7611),
 ('amp', 7297),
 ('like', 6997),
 ('out', 6980),
 ('get', 6917),
 ('just', 6895),
 ('if', 6825),
 ('or', 6676),
 ('now', 6290),
 ('our', 6260),
 ('who', 5677),
 ('do', 5531),
 ('more', 5522),
 ('up', 5348),
 ('time', 5253),
 ('i', 5168),
 ('no', 5059),
 ('been', 4911),
 ('

In [21]:
FREQWORDS = set([w for (w, wc) in cnt.most_common(70)])
def remove_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

In [22]:
twitter_data['tweets'] = twitter_data['tweets'].apply(lambda text: remove_freqwords(text))

In [23]:
twitter_data['tweets'].head(10)

0    Please guys adjusting realities You know oppos...
1    First Name CORONA Surname VIRUS NickName Age 1...
2    Liked YouTube Conversation Dad feat Jeetu Gajr...
3                    Letâ€™s fight Virus Get well soon
4    Aaj desh mein ko Phelne se Rokna bhut hi muski...
5    bought due FIFA shit Star Wars hard Iâ€™m shit...
6    First death due Buldana Maharashtra its Mahara...
7    How handle crisis determine place world future...
8                                shit fuckin sleep smh
9         doesnt weather rich poverty What ðŸŒ¹ single
Name: tweets, dtype: object

Remove rare words

In [24]:
n_rare_words = 100
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
def remove_rarewords(text):
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

In [25]:
RAREWORDS

{'100200',
 '2breatheasily',
 '365579',
 '6010this',
 '8010',
 'AoA',
 'BECAME',
 'Builds',
 'Castel',
 'ChillðŸ¤·ðŸ\x8f¾â€\x8dâ™‚ï¸\x8f',
 'CovidReport',
 'EUGEWJHWHR',
 'Fcking',
 'HHHHHHHHAAAAAAAAAAAAAAA',
 'Hmsan',
 'Intentions',
 'JustinBieber',
 'KeeptheRepublicSafe',
 'Kurasini',
 'Kyoto',
 'LOOOL',
 'Lemmon',
 'LytleðŸŽ‰ðŸŽ‰',
 'Marion',
 'Mightyraccoon',
 'Nammonia',
 'Natti',
 'NetflixandChill',
 'Roblox',
 'Samachar',
 'SummerWalker',
 'Sunnybrook',
 'Switches',
 'TJE',
 'Tumeric',
 'USaskatoon',
 'UToronto',
 'aroundðŸ¤¦â€\x8dâ™€ï¸\x8f',
 'asiaâ€™s',
 'asymmetry',
 'b33fday',
 'badhad',
 'blowsâ€\x9d',
 'broðŸ˜\xadðŸ˜\xad',
 'casesampoverwhelmed',
 'casesas',
 'cattyðŸ˜˜',
 'coronabriefing',
 'coronaplease',
 'coronataskforce',
 'cureUse',
 'devenezgoomba',
 'dismount',
 'drfauci',
 'easyTrump',
 'enor',
 'exceptionalismChurchill',
 'flem',
 'gZone',
 'hongjoong',
 'interrupting',
 'intothespiderverse',
 'inversion',
 'jinyoung',
 'ketu',
 'liarthe',
 'lolwut',
 'madeUP',
 

In [26]:
twitter_data['tweets'] = twitter_data['tweets'].apply(lambda text: remove_rarewords(text))

In [27]:
twitter_data['tweets'].head(10)

0    Please guys adjusting realities You know oppos...
1    First Name CORONA Surname VIRUS NickName Age 1...
2    Liked YouTube Conversation Dad feat Jeetu Gajr...
3                    Letâ€™s fight Virus Get well soon
4    Aaj desh mein ko Phelne se Rokna bhut hi muski...
5    bought due FIFA shit Star Wars hard Iâ€™m shit...
6    First death due Buldana Maharashtra its Mahara...
7    How handle crisis determine place world future...
8                                shit fuckin sleep smh
9         doesnt weather rich poverty What ðŸŒ¹ single
Name: tweets, dtype: object

Lemmatize

In [28]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

In [29]:
import nltk
nltk.download('book')

#nltk.download('all-corpora')

[nltk_data] Downloading collection 'book'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /Users/krish/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /Users/krish/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package chat80 to
[nltk_data]    |     /Users/krish/nltk_data...
[nltk_data]    |   Package chat80 is already up-to-date!
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/krish/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package conll2000 to
[nltk_data]    |     /Users/krish/nltk_data...
[nltk_data]    |   Package conll2000 is already up-to-date!
[nltk_data]    | Downloading package conll2002 to
[nltk_data]    |     /Users/krish/nltk_data...
[nltk_data]    |   Package conll2002 is already up-to-date!
[nltk_data]    | Downloading package dependency_t

True

In [30]:
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [31]:
twitter_data['tweets'] = twitter_data['tweets'].apply(lambda text: lemmatize_words(text))

In [32]:
lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}

In [33]:
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

In [34]:
twitter_data['tweets'] = twitter_data['tweets'].apply(lambda text: lemmatize_words(text))

In [35]:
twitter_data.head(10)

,Id,Timestamp,Source,Retwwet_Count,User_Name,tweets
0,1244239930341100032,2020-03-29 12:28:10,Twitter Web App,0,Mamaa,Please guy adjust reality You know opposed med...
1,1244239930123049984,2020-03-29 12:28:10,Twitter for Android,0,Opemi Dmh,First Name CORONA Surname VIRUS NickName Age 1...
2,1244239929619680000,2020-03-29 12:28:10,IFTTT,0,Muhammad Iqbal Dar,Liked YouTube Conversation Dad feat Jeetu Gajr...
3,1244239928919119872,2020-03-29 12:28:10,Twitter for iPhone,0,â™¥Dream 2021 ðŸ’™ Rache â™¥,Letâ€™s fight Virus Get well soon
4,1244239928608740096,2020-03-29 12:28:10,Twitter for Android,0,Alaxender0687,Aaj desh mein ko Phelne se Rokna bhut hi muski...
5,1244239927728110080,2020-03-29 12:28:10,Twitter for iPhone,0,Realo,buy due FIFA shit Star Wars hard Iâ€™m shit CO...
6,1244239926553530112,2020-03-29 12:28:10,Twitter for iPhone,0,Tauseef Shaikh,First death due Buldana Maharashtra it Maharas...
7,1244239922640240128,2020-03-29 12:28:09,Twitter for Android,0,Ashok Atluri,How handle crisis determine place world future...
8,1244239922158080000,2020-03-29 12:28:08,Twitter for iPhone,0,HUNCHOFMR ðŸ’°,shit fuckin sleep smh
9,1244239922090970112,2020-03-29 12:28:08,Twitter for Android,0,Jacob Adoo,doesnt weather rich poverty What ðŸŒ¹ single


Removal of Stopwords

In [36]:
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [37]:
STOPWORDS = set(stopwords.words('english'))

In [38]:
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

In [39]:
twitter_data['tweets'] = twitter_data['tweets'].apply(lambda text: remove_stopwords(text))

In [40]:
twitter_data['tweets'].head(10)

0    Please guy adjust reality You know opposed med...
1    First Name CORONA Surname VIRUS NickName Age 1...
2    Liked YouTube Conversation Dad feat Jeetu Gajr...
3                    Letâ€™s fight Virus Get well soon
4    Aaj desh mein ko Phelne se Rokna bhut hi muski...
5    buy due FIFA shit Star Wars hard Iâ€™m shit CO...
6    First death due Buldana Maharashtra Maharashtr...
7    How handle crisis determine place world future...
8                                shit fuckin sleep smh
9         doesnt weather rich poverty What ðŸŒ¹ single
Name: tweets, dtype: object

Removal of unwanted Emojis

In [41]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  
                           u"\U0001F300-\U0001F5FF"  
                           u"\U0001F680-\U0001F6FF" 
                           u"\U0001F1E0-\U0001F1FF" 
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [42]:
twitter_data['tweets'] = twitter_data['tweets'].apply(lambda text: remove_emoji(text))

In [43]:
twitter_data['tweets'].head(5)

0    Please guy adjust reality You know opposed med...
1    First Name CORONA Surname VIRUS NickName Age 1...
2    Liked YouTube Conversation Dad feat Jeetu Gajr...
3                    Letâ€™s fight Virus Get well soon
4    Aaj desh mein ko Phelne se Rokna bhut hi muski...
Name: tweets, dtype: object

********** Converting Emoticons to Words

In [44]:
#!pip install emoji

In [45]:
import emoji

*** Defining Emoticons

In [46]:
EMOTICONS = {
    u":‑\)":"Happy face or smiley",
    u":\)":"Happy face or smiley",
    u":-\]":"Happy face or smiley",
    u":\]":"Happy face or smiley",
    u":-3":"Happy face smiley",
    u":3":"Happy face smiley",
    u":->":"Happy face smiley",
    u":>":"Happy face smiley",
    u"8-\)":"Happy face smiley",
    u":o\)":"Happy face smiley",
    u":-\}":"Happy face smiley",
    u":\}":"Happy face smiley",
    u":-\)":"Happy face smiley",
    u":c\)":"Happy face smiley",
    u":\^\)":"Happy face smiley",
    u"=\]":"Happy face smiley",
    u"=\)":"Happy face smiley",
    u":‑D":"Laughing, big grin or laugh with glasses",
    u":D":"Laughing, big grin or laugh with glasses",
    u"8‑D":"Laughing, big grin or laugh with glasses",
    u"8D":"Laughing, big grin or laugh with glasses",
    u"X‑D":"Laughing, big grin or laugh with glasses",
    u"XD":"Laughing, big grin or laugh with glasses",
    u"=D":"Laughing, big grin or laugh with glasses",
    u"=3":"Laughing, big grin or laugh with glasses",
    u"B\^D":"Laughing, big grin or laugh with glasses",
    u":-\)\)":"Very happy",
    u":‑\(":"Frown, sad, andry or pouting",
    u":-\(":"Frown, sad, andry or pouting",
    u":\(":"Frown, sad, andry or pouting",
    u":‑c":"Frown, sad, andry or pouting",
    u":c":"Frown, sad, andry or pouting",
    u":‑<":"Frown, sad, andry or pouting",
    u":<":"Frown, sad, andry or pouting",
    u":‑\[":"Frown, sad, andry or pouting",
    u":\[":"Frown, sad, andry or pouting",
    u":-\|\|":"Frown, sad, andry or pouting",
    u">:\[":"Frown, sad, andry or pouting",
    u":\{":"Frown, sad, andry or pouting",
    u":@":"Frown, sad, andry or pouting",
    u">:\(":"Frown, sad, andry or pouting",
    u":'‑\(":"Crying",
    u":'\(":"Crying",
    u":'‑\)":"Tears of happiness",
    u":'\)":"Tears of happiness",
    u"D‑':":"Horror",
    u"D:<":"Disgust",
    u"D:":"Sadness",
    u"D8":"Great dismay",
    u"D;":"Great dismay",
    u"D=":"Great dismay",
    u"DX":"Great dismay",
    u":‑O":"Surprise",
    u":O":"Surprise",
    u":‑o":"Surprise",
    u":o":"Surprise",
    u":-0":"Shock",
    u"8‑0":"Yawn",
    u">:O":"Yawn",
    u":-\*":"Kiss",
    u":\*":"Kiss",
    u":X":"Kiss",
    u";‑\)":"Wink or smirk",
    u";\)":"Wink or smirk",
    u"\*-\)":"Wink or smirk",
    u"\*\)":"Wink or smirk",
    u";‑\]":"Wink or smirk",
    u";\]":"Wink or smirk",
    u";\^\)":"Wink or smirk",
    u":‑,":"Wink or smirk",
    u";D":"Wink or smirk",
    u":‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"X‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"XP":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"d:":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"=p":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u">:P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":-[.]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":S":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":‑\|":"Straight face",
    u":\|":"Straight face",
    u":$":"Embarrassed or blushing",
    u":‑x":"Sealed lips or wearing braces or tongue-tied",
    u":x":"Sealed lips or wearing braces or tongue-tied",
    u":‑#":"Sealed lips or wearing braces or tongue-tied",
    u":#":"Sealed lips or wearing braces or tongue-tied",
    u":‑&":"Sealed lips or wearing braces or tongue-tied",
    u":&":"Sealed lips or wearing braces or tongue-tied",
    u"O:‑\)":"Angel, saint or innocent",
    u"O:\)":"Angel, saint or innocent",
    u"0:‑3":"Angel, saint or innocent",
    u"0:3":"Angel, saint or innocent",
    u"0:‑\)":"Angel, saint or innocent",
    u"0:\)":"Angel, saint or innocent",
    u":‑b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"0;\^\)":"Angel, saint or innocent",
    u">:‑\)":"Evil or devilish",
    u">:\)":"Evil or devilish",
    u"\}:‑\)":"Evil or devilish",
    u"\}:\)":"Evil or devilish",
    u"3:‑\)":"Evil or devilish",
    u"3:\)":"Evil or devilish",
    u">;\)":"Evil or devilish",
    u"\|;‑\)":"Cool",
    u"\|‑O":"Bored",
    u":‑J":"Tongue-in-cheek",
    u"#‑\)":"Party all night",
    u"%‑\)":"Drunk or confused",
    u"%\)":"Drunk or confused",
    u":-###..":"Being sick",
    u":###..":"Being sick",
    u"<:‑\|":"Dump",
    u"\(>_<\)":"Troubled",
    u"\(>_<\)>":"Troubled",
    u"\(';'\)":"Baby",
    u"\(\^\^>``":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(\^_\^;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(~_~;\) \(・\.・;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-\)zzz":"Sleeping",
    u"\(\^_-\)":"Wink",
    u"\(\(\+_\+\)\)":"Confused",
    u"\(\+o\+\)":"Confused",
    u"\(o\|o\)":"Ultraman",
    u"\^_\^":"Joyful",
    u"\(\^_\^\)/":"Joyful",
    u"\(\^O\^\)／":"Joyful",
    u"\(\^o\^\)／":"Joyful",
    u"\(__\)":"Kowtow as a sign of respect, or dogeza for apology",
    u"_\(\._\.\)_":"Kowtow as a sign of respect, or dogeza for apology",
    u"<\(_ _\)>":"Kowtow as a sign of respect, or dogeza for apology",
    u"<m\(__\)m>":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(__\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(_ _\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"\('_'\)":"Sad or Crying",
    u"\(/_;\)":"Sad or Crying",
    u"\(T_T\) \(;_;\)":"Sad or Crying",
    u"\(;_;":"Sad of Crying",
    u"\(;_:\)":"Sad or Crying",
    u"\(;O;\)":"Sad or Crying",
    u"\(:_;\)":"Sad or Crying",
    u"\(ToT\)":"Sad or Crying",
    u";_;":"Sad or Crying",
    u";-;":"Sad or Crying",
    u";n;":"Sad or Crying",
    u";;":"Sad or Crying",
    u"Q\.Q":"Sad or Crying",
    u"T\.T":"Sad or Crying",
    u"QQ":"Sad or Crying",
    u"Q_Q":"Sad or Crying",
    u"\(-\.-\)":"Shame",
    u"\(-_-\)":"Shame",
    u"\(一一\)":"Shame",
    u"\(；一_一\)":"Shame",
    u"\(=_=\)":"Tired",
    u"\(=\^\·\^=\)":"cat",
    u"\(=\^\·\·\^=\)":"cat",
    u"=_\^=	":"cat",
    u"\(\.\.\)":"Looking down",
    u"\(\._\.\)":"Looking down",
    u"\^m\^":"Giggling with hand covering mouth",
    u"\(\・\・?":"Confusion",
    u"\(?_?\)":"Confusion",
    u">\^_\^<":"Normal Laugh",
    u"<\^!\^>":"Normal Laugh",
    u"\^/\^":"Normal Laugh",
    u"\（\*\^_\^\*）" :"Normal Laugh",
    u"\(\^<\^\) \(\^\.\^\)":"Normal Laugh",
    u"\(^\^\)":"Normal Laugh",
    u"\(\^\.\^\)":"Normal Laugh",
    u"\(\^_\^\.\)":"Normal Laugh",
    u"\(\^_\^\)":"Normal Laugh",
    u"\(\^\^\)":"Normal Laugh",
    u"\(\^J\^\)":"Normal Laugh",
    u"\(\*\^\.\^\*\)":"Normal Laugh",
    u"\(\^—\^\）":"Normal Laugh",
    u"\(#\^\.\^#\)":"Normal Laugh",
    u"\（\^—\^\）":"Waving",
    u"\(;_;\)/~~~":"Waving",
    u"\(\^\.\^\)/~~~":"Waving",
    u"\(-_-\)/~~~ \($\·\·\)/~~~":"Waving",
    u"\(T_T\)/~~~":"Waving",
    u"\(ToT\)/~~~":"Waving",
    u"\(\*\^0\^\*\)":"Excited",
    u"\(\*_\*\)":"Amazed",
    u"\(\*_\*;":"Amazed",
    u"\(\+_\+\) \(@_@\)":"Amazed",
    u"\(\*\^\^\)v":"Laughing,Cheerful",
    u"\(\^_\^\)v":"Laughing,Cheerful",
    u"\(\(d[-_-]b\)\)":"Headphones,Listening to music",
    u'\(-"-\)':"Worried",
    u"\(ーー;\)":"Worried",
    u"\(\^0_0\^\)":"Eyeglasses",
    u"\(\＾ｖ\＾\)":"Happy",
    u"\(\＾ｕ\＾\)":"Happy",
    u"\(\^\)o\(\^\)":"Happy",
    u"\(\^O\^\)":"Happy",
    u"\(\^o\^\)":"Happy",
    u"\)\^o\^\(":"Happy",
    u":O o_O":"Surprised",
    u"o_0":"Surprised",
    u"o\.O":"Surpised",
    u"\(o\.o\)":"Surprised",
    u"oO":"Surprised",
    u"\(\*￣m￣\)":"Dissatisfied",
    u"\(‘A`\)":"Snubbed or Deflated"
}

In [47]:
def convert_emoticons(text):
    for emot in EMOTICONS:
        text = re.sub(u'('+emot+')', "_".join(EMOTICONS[emot].replace(",","").split()), text)
    return text

In [48]:
twitter_data['tweets'] = twitter_data['tweets'].apply(lambda text: convert_emoticons(text))

In [49]:
twitter_data['tweets'].head(5)

0    Please guy adjust reality You know opposed med...
1    First Name CORONA Surname VIRUS NickName Age 1...
2    Liked YouTube Conversation Dad feat Jeetu Gajr...
3                    Letâ€™s fight Virus Get well soon
4    Aaj desh mein ko Phelne se Rokna bhut hi muski...
Name: tweets, dtype: object

Converting Chat Words

**** Defining the Dictionary

In [50]:
chat_words_str = """
AFAIK=As Far As I Know
AFK=Away From Keyboard
ASAP=As Soon As Possible
ATK=At The Keyboard
ATM=At The Moment
A3=Anytime, Anywhere, Anyplace
BAK=Back At Keyboard
BBL=Be Back Later
BBS=Be Back Soon
BFN=Bye For Now
B4N=Bye For Now
BRB=Be Right Back
BRT=Be Right There
BTW=By The Way
B4=Before
B4N=Bye For Now
CU=See You
CUL8R=See You Later
CYA=See You
FAQ=Frequently Asked Questions
FC=Fingers Crossed
FWIW=For What It's Worth
FYI=For Your Information
GAL=Get A Life
GG=Good Game
GN=Good Night
GMTA=Great Minds Think Alike
GR8=Great!
G9=Genius
IC=I See
ICQ=I Seek you (also a chat program)
ILU=ILU: I Love You
IMHO=In My Honest/Humble Opinion
IMO=In My Opinion
IOW=In Other Words
IRL=In Real Life
KISS=Keep It Simple, Stupid
LDR=Long Distance Relationship
LMAO=Laugh My A.. Off
LOL=Laughing Out Loud
LTNS=Long Time No See
L8R=Later
MTE=My Thoughts Exactly
M8=Mate
NRN=No Reply Necessary
OIC=Oh I See
PITA=Pain In The A..
PRT=Party
PRW=Parents Are Watching
ROFL=Rolling On The Floor Laughing
ROFLOL=Rolling On The Floor Laughing Out Loud
ROTFLMAO=Rolling On The Floor Laughing My A.. Off
SK8=Skate
STATS=Your sex and age
ASL=Age, Sex, Location
THX=Thank You
TTFN=Ta-Ta For Now!
TTYL=Talk To You Later
U=You
U2=You Too
U4E=Yours For Ever
WB=Welcome Back
WTF=What The F...
WTG=Way To Go!
WUF=Where Are You From?
W8=Wait...
7K=Sick:-D Laugher
"""

In [51]:
chat_words_map_dict = {}
chat_words_list = []

In [52]:
for line in chat_words_str.split("\n"):
    if line != "":
        cw = line.split("=")[0]
        cw_expanded = line.split("=")[1]
        chat_words_list.append(cw)
        chat_words_map_dict[cw] = cw_expanded
chat_words_list = set(chat_words_list)

In [53]:
def chat_words_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words_list:
            new_text.append(chat_words_map_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

In [54]:
twitter_data['tweets'] = twitter_data['tweets'].apply(lambda text: chat_words_conversion(text))

In [55]:
twitter_data['tweets'].head(10)

0    Please guy adjust reality You know opposed med...
1    First Name CORONA Surname VIRUS NickName Age 1...
2    Liked YouTube Conversation Dad feat Jeetu Gajr...
3                    Letâ€™s fight Virus Get well soon
4    Aaj desh mein ko Phelne se Rokna bhut hi muski...
5    buy due FIFA shit Star Wars hard Iâ€™m shit CO...
6    First death due Buldana Maharashtra Maharashtr...
7    How handle crisis determine place world future...
8                                shit fuckin sleep smh
9         doesnt weather rich poverty What ðŸŒ¹ single
Name: tweets, dtype: object

Some more Cleansing using RE Library

In [56]:
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    text = text.lower()       
    return text

In [57]:
twitter_data['tweets'] = twitter_data['tweets'].apply(lambda text: clean_text(text))

In [58]:
twitter_data['tweets'].head(10)

0    please guy adjust reality you know opposed med...
1    first name corona surname virus nickname age  ...
2    liked youtube conversation dad feat jeetu gajr...
3                    let   s fight virus get well soon
4    aaj desh mein ko phelne se rokna bhut hi muski...
5    buy due fifa shit star wars hard i   m shit co...
6    first death due buldana maharashtra maharashtr...
7    how handle crisis determine place world future...
8                                shit fuckin sleep smh
9         doesnt weather rich poverty what      single
Name: tweets, dtype: object

In [59]:
num_words = twitter_data['tweets'].apply(lambda x: len(x.split()))
num_words_mean, num_words_std = np.mean(num_words), np.std(num_words)

num_sentences = twitter_data['tweets'].apply(lambda x: len(re.split( '~ ...' ,'~'.join(x.split('.')))))
num_sentences_mean = np.mean(num_sentences)

twitter_data['tweets'] = twitter_data['tweets'].str.replace(r'twitter',r' ')
twitter_data['tweets'] = twitter_data['tweets'].str.replace(r'padmaja',r' ')
twitter_data['tweets'] = twitter_data['tweets'].str.replace(r'joshi',r' ')
twitter_data['tweets'] = twitter_data['tweets'].str.replace(r'please',r' ')
twitter_data['tweets'] = twitter_data['tweets'].str.replace(r'healthkart',r' ')

Remove extra spaces and Words less than 3 letters

In [60]:
twitter_data['tweets'] = twitter_data['tweets'].str.replace(r'[ ]{2, }',r' ')
twitter_data['tweets'] = twitter_data['tweets'].str.replace(r'\b\w{1,3}\b', r'')

In [61]:
twitter_data.head(10)

,Id,Timestamp,Source,Retwwet_Count,User_Name,tweets
0,1244239930341100032,2020-03-29 12:28:10,Twitter Web App,0,Mamaa,adjust reality know opposed medium tour w...
1,1244239930123049984,2020-03-29 12:28:10,Twitter for Android,0,Opemi Dmh,first name corona surname virus nickname n...
2,1244239929619680000,2020-03-29 12:28:10,IFTTT,0,Muhammad Iqbal Dar,liked youtube conversation feat jeetu gajraj
3,1244239928919119872,2020-03-29 12:28:10,Twitter for iPhone,0,â™¥Dream 2021 ðŸ’™ Rache â™¥,fight virus well soon
4,1244239928608740096,2020-03-29 12:28:10,Twitter for Android,0,Alaxender0687,desh mein phelne rokna bhut muskil hbecaus...
5,1244239927728110080,2020-03-29 12:28:10,Twitter for iPhone,0,Realo,fifa shit star wars hard shit guess
6,1244239926553530112,2020-03-29 12:28:10,Twitter for iPhone,0,Tauseef Shaikh,first death buldana maharashtra maharashtra s...
7,1244239922640240128,2020-03-29 12:28:09,Twitter for Android,0,Ashok Atluri,handle crisis determine place world future ex...
8,1244239922158080000,2020-03-29 12:28:08,Twitter for iPhone,0,HUNCHOFMR ðŸ’°,shit fuckin sleep
9,1244239922090970112,2020-03-29 12:28:08,Twitter for Android,0,Jacob Adoo,doesnt weather rich poverty what single


In [62]:
twitter_data['text_length'] = [len(text.split(' ')) for text in twitter_data.tweets]
print(twitter_data.shape)

(126031, 7)


Dropping texts with length <3 and drop duplicates

In [63]:
twitter_data = twitter_data[twitter_data['text_length']>2]
twitter_data = twitter_data.drop_duplicates(subset=['tweets'])

print(twitter_data.shape)

(89715, 7)


In [64]:
twitter_data.head(10)

,Id,Timestamp,Source,Retwwet_Count,User_Name,tweets,text_length
0,1244239930341100032,2020-03-29 12:28:10,Twitter Web App,0,Mamaa,adjust reality know opposed medium tour w...,14
1,1244239930123049984,2020-03-29 12:28:10,Twitter for Android,0,Opemi Dmh,first name corona surname virus nickname n...,34
2,1244239929619680000,2020-03-29 12:28:10,IFTTT,0,Muhammad Iqbal Dar,liked youtube conversation feat jeetu gajraj,8
3,1244239928919119872,2020-03-29 12:28:10,Twitter for iPhone,0,â™¥Dream 2021 ðŸ’™ Rache â™¥,fight virus well soon,9
4,1244239928608740096,2020-03-29 12:28:10,Twitter for Android,0,Alaxender0687,desh mein phelne rokna bhut muskil hbecaus...,43
5,1244239927728110080,2020-03-29 12:28:10,Twitter for iPhone,0,Realo,fifa shit star wars hard shit guess,17
6,1244239926553530112,2020-03-29 12:28:10,Twitter for iPhone,0,Tauseef Shaikh,first death buldana maharashtra maharashtra s...,12
7,1244239922640240128,2020-03-29 12:28:09,Twitter for Android,0,Ashok Atluri,handle crisis determine place world future ex...,10
8,1244239922158080000,2020-03-29 12:28:08,Twitter for iPhone,0,HUNCHOFMR ðŸ’°,shit fuckin sleep,4
9,1244239922090970112,2020-03-29 12:28:08,Twitter for Android,0,Jacob Adoo,doesnt weather rich poverty what single,11


In [65]:
twitter_data.to_csv('Compiled & Processed Data.csv')